We didn't understand the problem, so this model predicted the GDP growth for the same year

In [1]:
import sys
import time
sys.path.append("..")
import utils
import utils.config
import pandas as pd
import sqlite3
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [2]:
conn = sqlite3.connect('../db.sqlite3')

indicators = utils.config.INDICATORS
target = utils.config.TARGET
separator = "',\n'"
sql_string = f""" SELECT
                t2.Country,
                t1.Year,
                t3.IndicatorName,
                t1.Value                
                FROM CountryIndicators t1
                LEFT JOIN
                (SELECT ShortName as Country, CountryCode from Countries) t2
                ON t1.CountryCode = t2.CountryCode
                LEFT JOIN
                (SELECT IndicatorCode, IndicatorName from Indicators)t3
                ON t1.IndicatorCode = t3.IndicatorCode
                WHERE t3.IndicatorName in ('{separator.join(indicators)}');"""

raw_df = pd.read_sql(sql_string,conn)

In [107]:
df = raw_df.groupby(["Country","Year","IndicatorName"]).sum().squeeze().unstack()
df = df.reset_index(level=["Country","Year"])

features = df.columns.tolist()
# Put "Year" as the last list element
features.remove("Year")
features.append("Year")

df = df.groupby(["Country","Year"]).sum().squeeze().unstack()
df = df.reset_index(level=["Country"])

df = df.dropna()
df

IndicatorName     Country GDP growth (annual %)                        \
Year                                       1961       1962       1963   
2                 Algeria            -13.605441 -19.685042  34.313729   
7               Argentina              5.427843  -0.852022  -5.308197   
10              Australia              2.475929   1.327575   6.212444   
11                Austria              5.537979   2.648675   4.138268   
14             Bangladesh              6.058161   5.453031  -0.455894   
..                    ...                   ...        ...        ...   
227               Uruguay              2.493038  -1.574549   0.170459   
230             Venezuela              3.192519   8.532933   3.900951   
234                 World              4.289940   5.557479   5.211821   
236                Zambia              1.361382  -2.490840   3.272393   
237              Zimbabwe              6.316157   1.434471   6.244345   

IndicatorName                                                                  \
Year                1964       1965      1966      1967       1968       1969   
2               5.839413   6.206898 -4.804971  9.452963  10.796239   8.433280   
7              10.130298  10.569433 -0.659726  3.191997   4.822501   9.679526   
10              6.976155   5.980020  2.365193  6.297056   5.092821   7.049712   
11              6.124354   3.480175  5.642861  3.008048   4.472313   6.275867   
14             10.952789   1.606258  2.566812 -1.875864   9.489454   1.220858   
..                   ...        ...       ...       ...        ...        ...   
227             2.440521   1.045604  3.062856 -3.656834   1.888912   5.864741   
230            11.129345   4.162867  1.510250  2.833869   7.337233   0.706037   
234             6.601864   5.536151  5.834650  4.393750   6.070893   5.868154   
236            12.214048  16.647456 -5.570310  7.919696   1.248330  -0.436916   
237            -1.106172   4.910571  1.523130  8.367009   1.970135  12.428236   

IndicatorName  ... GDP per capita growth (annual %)                        \
Year           ...                             2001       2002       2003   
2              ...                         3.265847   4.279382   5.861446   
7              ...                        -5.465177 -11.877289   7.643732   
10             ...                         0.564238   2.596754   1.806724   
11             ...                         0.963289   1.156956   0.266533   
14             ...                         3.117281   1.963914   2.947270   
..             ...                              ...        ...        ...   
227            ...                        -4.013575  -7.750445   0.869938   
230            ...                         1.492421 -10.502473  -9.390640   
234            ...                         0.524248   0.795259   1.477050   
236            ...                         2.640438   1.891107   4.267813   
237            ...                         0.602544  -9.521737 -17.532650   

IndicatorName                                                                \
Year                2004      2005      2006      2007       2008      2009   
2               2.957718  4.465430  0.249229  1.852883   0.391115 -0.095366   
7               7.849148  8.036045  7.270415  6.854978   2.023423 -0.972043   
10              2.948889  1.853481  1.474789  3.111526   1.649081 -0.257393   
11              2.070516  1.447197  2.840717  3.286169   1.229876 -4.050747   
14              3.563089  4.980956  5.265728  5.778069   4.826894  3.886546   
..                   ...       ...       ...       ...        ...       ...   
227             5.052839  7.411275  3.928790  6.263685   6.821688  3.873703   
230            16.232654  8.444335  8.047300  6.987073   3.607786 -4.700066   
234             2.852738  2.310779  2.811061  2.679578   0.217597 -3.247018   
236             4.297361  4.404533  4.958771  5.314451   4.684774  6.040487   
237            -6.493173 -6.557725 -4.516923 -

In [158]:
country_list = df.Country.unique()
#country_list = ["United Kingdom"]
X = []
y = []
for country in country_list:
    df_country = df.loc[df["Country"] == country]
    for year in range(45):
        dataset = []
        for i, _ in enumerate(indicators):
            for j in range(5):
                dataset.append(float(df_country[indicators[i]].iloc[:,year+j].to_numpy()))
        X.append(dataset)
        y.append(float(df_country[target].iloc[:,year+5].to_numpy()))

In [95]:
country_list = df.Country.unique()[:10]
#country_list = ["United Kingdom"]
X = []
y = []
for k, country in enumerate(country_list):
    for year in range(45):
        dataset = []
        for i, _ in enumerate(indicators):
            for j in range(5):
                dataset.append(k**year*(8*i+j))
        X.append(dataset)
        y.append(k**year*(8*i+j+1))

In [159]:
print(X[-1])
print()
print(y[-1])

[-5.7110838330468, -3.46149505040178, -3.65332704339563, -17.6689465976187, 5.98439075352977, -6.55772514849929, -4.5169226554587, -4.88398647149923, -18.8748249395182, 4.24230236384516]

11.3759212337497


In [160]:
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (3465, 10)
X_test shape: (1485, 10)
y_train shape: (3465,)
y_test shape: (1485,)


In [161]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_y_pred = linear_model.predict(X_test)

results_df = pd.DataFrame()
results_df["y_real"] = y_test
results_df["y_pred"] = linear_y_pred
results_df["err"] = np.absolute(results_df["y_real"] - results_df["y_pred"])
#results_df["%_err"] = ((results_df["err"]) /(np.absolute (results_df["y_real"]))* 100)

print(f"RMSE: {mean_squared_error(y_test, linear_y_pred)**0.5}")
print(f"R^2: {r2_score(y_test, linear_y_pred)}")

results_df

RMSE: 5.037190245567862
R^2: 0.09355592568298543


,y_real,y_pred,err
0,-13.126724,4.287967,17.414691
1,5.110595,3.775012,1.335583
2,5.104300,3.852555,1.251745
3,2.073577,2.948719,0.875142
4,2.135375,3.012702,0.877327
...,...,...,...
1480,6.109318,4.502685,1.606633
1481,7.703461,4.555717,3.147744
1482,9.645883,5.081580,4.564303
1483,5.462679,4.911669,0.551010


In [162]:
gbm_hyperparams = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'loss': 'ls'
}
gbm_model = GradientBoostingRegressor(**gbm_hyperparams)

t0 = time.time()
gbm_model.fit(X_train, y_train)
print(f"Elapsed time training: {time.time() - t0} seconds")
t0 = time.time()
gbm_y_pred = gbm_model.predict(X_test)
print(f"Elapsed time predicting: {time.time() - t0} seconds")

results_df = pd.DataFrame()
results_df["y_real"] = y_test
results_df["y_pred"] = gbm_y_pred
results_df["err"] = np.absolute(results_df["y_real"] - results_df["y_pred"])
results_df["%_err"] = ((results_df["err"]) /(np.absolute (results_df["y_real"]))* 100)

print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

results_df

Elapsed time training: 1.963604211807251 seconds
Elapsed time predicting: 0.004141569137573242 seconds
RMSE: 5.947189329936022
R^2: -0.2635366280067297


,y_real,y_pred,err,%_err
0,-13.126724,4.094545,17.221269,131.192434
1,5.110595,3.098904,2.011691,39.363147
2,5.104300,3.178365,1.925935,37.731615
3,2.073577,1.464326,0.609251,29.381655
4,2.135375,1.675688,0.459687,21.527219
...,...,...,...,...
1480,6.109318,3.591558,2.517760,41.211802
1481,7.703461,5.379487,2.323974,30.167921
1482,9.645883,4.942275,4.703608,48.762858
1483,5.462679,5.998712,0.536033,9.812633
